In [ ]:
#@title <font size="6" color="orange">FramePack Launcher</font>
#@markdown ### Click the **public URL** (e.g., `https://xxxxxxx.gradio.live`) to open the app.

use_framepack_f1 = False  #@param {type: "boolean"}
NGROK_TOKEN = '2uwbSFzPCb2kEfaxkHzrvrqmvCn_2gQjxVMynAKL5dcAnTmmQ'  #@param {type: "string"}

def start_ngrok(port):
    %pip install pyngrok
    import time, socket
    while True:
        time.sleep(0.5)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            if sock.connect_ex(('127.0.0.1', port)) == 0:
                break
    config = f'''version: 3
endpoints:
  - name: gradio
    upstream:
      url: http://localhost:{port}
    traffic_policy:
      inbound:
        - name: "Remove X-Forwarded-Host"
          actions:
            - type: remove-headers
              config:
                headers:
                  - "x-forwarded-host"
agent:
  authtoken: {NGROK_TOKEN}
'''
    with open("/content/ngrok_config.yaml", "w") as f:
        f.write(config)
    !ngrok start gradio --config /content/ngrok_config.yaml --log=stdout

def setup_python_310():
    !python --version > /content/pyversion
    with open('/content/pyversion') as f:
        if '3.10' in f.read():
            return
    !apt-get update -y
    !apt-get install -y python3.10 python3.10-distutils
    !rm -f /usr/local/bin/python /usr/local/bin/pip
    !update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
    !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def clear_output_screen():
    from IPython.display import clear_output
    clear_output()

from google.colab import userdata
try:
    NGROK_TOKEN = userdata.get('NGROK')
    print("Loaded NGROK token from secrets.")
except:
    pass

setup_python_310()

%cd /content/
!git clone https://github.com/lllyasviel/FramePack
%cd FramePack
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install -r requirements.txt
!pip install sageattention==1.0.6

command = 'PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True'
command += ' python demo_gradio_f1.py' if use_framepack_f1 else ' python demo_gradio.py'

clear_output_screen()

if NGROK_TOKEN:
    import threading
    port = 7860
    threading.Thread(target=start_ngrok, args=(port,), daemon=True).start()
else:
    command += ' --share'

print(command)
!{command}